In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import joblib
import os

# 1) Load preprocessed data
X = np.load("../data/X.npy")
y = np.load("../data/y.npy")

print("Loaded data:")
print("X shape:", X.shape)
print("y shape:", y.shape)

# 2) Train/Test split (stratified)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("\nTraining set:", X_train.shape)
print("Testing set:", X_test.shape)

# 3) Build Neural Network Model

input_dim = X_train.shape[1]
print("Input dimensions:", input_dim)

model = Sequential([
    Dense(64, activation='relu', input_shape=(input_dim,)),
    Dropout(0.3),

    Dense(32, activation='relu'),
    Dropout(0.2),

    Dense(1, activation='sigmoid')  # binary output
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

# 4) Training

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

history = model.fit(
    X_train, y_train,
    epochs=30,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)



ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# 5) Evaluation
y_pred = (model.predict(X_test) > 0.5).astype(int)

print("\nAccuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# 6) Save the trained model
os.makedirs("../models", exist_ok=True)
model.save("../models/nslkdd_dnn_model.h5")

print("\nModel saved to: models/nslkdd_dnn_model.h5")

# 7) Save training history 
np.save("../models/training_history.npy", history.history)
print("History saved to models/training_history.npy")
